In [1]:
import pandas as pd
import numpy as np
import requests

In [5]:
# df = pd.read_csv('../data/extracted_data.csv')
df[['chr', 'start', 'id', 'ref', 'alt']]

,chr,start,id,ref,alt
0,1,1052729,mut1831,T,A
1,1,1227399,mut2068,G,A
2,1,1234820,mut1462,C,G
3,1,1256438,mut1952,C,T
4,1,1342375,mut943,A,G
...,...,...,...,...,...
3100,X,153006175,mut742,T,G
3101,X,153174592,mut1820,C,T
3102,X,153207408,mut1772,T,G
3103,X,153698355,mut1225,C,T


In [27]:
data_str = '{ "variants" : ['
for index, row in df.iterrows():
    data_str = data_str + '"{}  {}  {}  {}  {}  .   .   .",'.format(row['chr'], row['start'], row['id'], row['ref'], row['alt'])

    if index > 290:
        break
data_str = data_str[:-1]
data_str = data_str + ']}'

In [16]:
data_str

'{ "variants" : ["1  1052729  mut1831  T  A  .   .   .","1  1227399  mut2068  G  A  .   .   .","1  1234820  mut1462  C  G  .   .   .","1  1256438  mut1952  C  T  .   .   .","1  1342375  mut943  A  G  .   .   .","1  1549750  mut1281  C  A  .   .   .","1  2267093  test30  C  T  .   .   ."]}'

In [28]:
import requests, sys
 
server = "https://grch37.rest.ensembl.org"
ext = "/vep/homo_sapiens/region?GO=True&SpliceAI=True&dbscSNV=True"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}
r = requests.post(server+ext, headers=headers, data=data_str) # '{ "variants" : ["21  26960070  rs116645811 G A . . .", "21  26965148  rs1135638 G A . . ." ] }'
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
print(repr(decoded))

[{'input': '1 1052729 mut1831 T A . . .', 'seq_region_name': '1', 'end': 1052729, 'strand': 1, 'assembly_name': 'GRCh37', 'most_severe_consequence': 'upstream_gene_variant', 'start': 1052729, 'regulatory_feature_consequences': [{'consequence_terms': ['regulatory_region_variant'], 'regulatory_feature_id': 'ENSR00000000136', 'variant_allele': 'A', 'biotype': 'promoter', 'impact': 'MODIFIER'}], 'transcript_consequences': [{'biotype': 'protein_coding', 'gene_symbol': 'C1orf159', 'impact': 'MODIFIER', 'gene_id': 'ENSG00000131591', 'transcript_id': 'ENST00000294576', 'distance': 993, 'consequence_terms': ['upstream_gene_variant'], 'variant_allele': 'A', 'hgnc_id': 26062, 'strand': -1, 'gene_symbol_source': 'HGNC'}, {'gene_id': 'ENSG00000131591', 'gene_symbol': 'C1orf159', 'impact': 'MODIFIER', 'biotype': 'nonsense_mediated_decay', 'transcript_id': 'ENST00000379325', 'distance': 1254, 'consequence_terms': ['upstream_gene_variant'], 'hgnc_id': 26062, 'variant_allele': 'A', 'go': [{'description

In [30]:
df1 = pd.DataFrame(decoded)
df1

,input,seq_region_name,end,strand,assembly_name,most_severe_consequence,start,regulatory_feature_consequences,transcript_consequences,id,allele_string,motif_feature_consequences,colocated_variants,intergenic_consequences
0,1 1052729 mut1831 T A . . .,1,1052729,1,GRCh37,upstream_gene_variant,1052729,[{'consequence_terms': ['regulatory_region_var...,"[{'biotype': 'protein_coding', 'gene_symbol': ...",mut1831,T/A,NaN,NaN,NaN
1,1 1227399 mut2068 G A . . .,1,1227399,1,GRCh37,3_prime_UTR_variant,1227399,"[{'variant_allele': 'A', 'biotype': 'CTCF_bind...",[{'consequence_terms': ['3_prime_UTR_variant']...,mut2068,G/A,"[{'motif_pos': 1, 'high_inf_pos': 'N', 'impact...","[{'seq_region_name': '1', 'end': 1227399, 'sta...",NaN
2,1 1234820 mut1462 C G . . .,1,1234820,1,GRCh37,non_coding_transcript_exon_variant,1234820,"[{'regulatory_feature_id': 'ENSR00000918547', ...","[{'hgnc_id': 16754, 'variant_allele': 'G', 'st...",mut1462,C/G,NaN,"[{'seq_region_name': '1', 'end': 1234820, 'sta...",NaN
3,1 1256438 mut1952 C T . . .,1,1256438,1,GRCh37,stop_gained,1256438,NaN,"[{'transcript_id': 'ENST00000323275', 'cdna_st...",mut1952,C/T,NaN,"[{'somatic': 1, 'phenotype_or_disease': 1, 'al...",NaN
4,1 1342375 mut943 A G . . .,1,1342375,1,GRCh37,missense_variant,1342375,"[{'regulatory_feature_id': 'ENSR00000000188', ...","[{'sift_prediction': 'deleterious', 'gene_symb...",mut943,A/G,NaN,"[{'seq_region_name': '1', 'end': 1342375, 'sta...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,10 77161529 test416 G C . . .,10,77161529,1,GRCh37,5_prime_UTR_variant,77161529,[{'consequence_terms': ['regulatory_region_var...,"[{'strand': -1, 'gene_symbol_source': 'HGNC', ...",test416,G/C,NaN,"[{'id': 'COSV65307725', 'allele_string': 'COSM...",NaN
285,10 77161597 test347 G C . . .,10,77161597,1,GRCh37,5_prime_UTR_variant,77161597,[{'consequence_terms': ['regulatory_region_var...,"[{'gene_id': 'ENSG00000165655', 'spliceai': {'...",test347,G/C,NaN,"[{'id': 'COSV65307453', 'allele_string': 'COSM...",NaN
286,10 77161729 test489 C G . . .,10,77161729,1,GRCh37,non_coding_transcript_exon_variant,77161729,"[{'variant_allele': 'G', 'biotype': 'promoter'...","[{'gene_id': 'ENSG00000165655', 'biotype': 'pr...",test489,C/G,NaN,NaN,NaN
287,10 77161777 test344 C G . . .,10,77161777,1,GRCh37,non_coding_transcript_exon_variant,77161777,[{'consequence_terms': ['regulatory_region_var...,"[{'gene_id': 'ENSG00000165655', 'impact': 'MOD...",test344,C/G,NaN,NaN,NaN


In [41]:
df1['transcript_consequences'][1]

[{'consequence_terms': ['3_prime_UTR_variant'],
  'cdna_start': 2674,
  'transcript_id': 'ENST00000325425',
  'gene_id': 'ENSG00000162572',
  'gene_symbol': 'SCNN1D',
  'impact': 'MODIFIER',
  'cdna_end': 2674,
  'biotype': 'protein_coding',
  'gene_symbol_source': 'HGNC',
  'strand': 1,
  'go': [{'go_term': 'GO:0005272', 'description': 'sodium_channel_activity'},
   {'go_term': 'GO:0005515', 'description': 'protein_binding'},
   {'go_term': 'GO:0005886', 'description': 'plasma_membrane'},
   {'description': 'sodium_ion_transport', 'go_term': 'GO:0006814'},
   {'description': 'ligand-gated_sodium_channel_activity',
    'go_term': 'GO:0015280'},
   {'go_term': 'GO:0016020', 'description': 'membrane'},
   {'go_term': 'GO:0016021', 'description': 'integral_component_of_membrane'},
   {'description': 'ion_transmembrane_transport', 'go_term': 'GO:0034220'},
   {'go_term': 'GO:0050896', 'description': 'response_to_stimulus'},
   {'go_term': 'GO:0050909', 'description': 'sensory_perception_of